## Hyperparameter tuning via Optuna

### Being a binary model this notebook will be limited to predicting one class 1 or 0, yes or no.
### Here I will be predicting if a cell received a treatment or not

In [1]:
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import plotly
import seaborn as sns
import toml
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import preprocessing

from sklearn.model_selection import train_test_split

sys.path.append("..")
from MLP_utils.parameters import Parameters
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_auc_score

from sklearn.metrics import roc_curve, auc

import pyarrow.parquet as pq

from MLP_utils.utils import (
    Dataset_formatter,
    data_split,
    extract_best_trial_params,
    objective_model_optimizer,
    optimized_model_create,
    plot_metric_vs_epoch,
    results_output,
    test_optimized_model,
    train_optimized_model,
    un_nest,
)

from utils.utils import df_stats
from MLP_utils.utils import parameter_set

In [2]:
# Import Data
# set data file path under pathlib path for multi-system use
file_path = Path("../../Extracted_Features_(CSV_files)/feature_df_sc_norm.parquet")

df = pq.read_table(file_path).to_pandas()
# df = pd.read_csv(file_path, engine="pyarrow")

In [3]:
data = Path("MLP_utils/config.toml")
config = toml.load(data)
params = Parameters()
params = parameter_set(params, config)

#### Set up Data to be compatible with model

##### Classification Models:
Comment out code if using regression

In [4]:
df

,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,Metadata_inducer1,Metadata_inducer1_concentration,Metadata_inducer1_concentration_unit,...,Nuclei_Texture_Variance_CorrPM_3_00_256,Nuclei_Texture_Variance_CorrPM_3_01_256,Nuclei_Texture_Variance_CorrPM_3_02_256,Nuclei_Texture_Variance_CorrPM_3_03_256,Metadata_Treatment,Metadata_Dose,oneb_Metadata_Treatment_Dose_Inhibitor_Dose,twob_Metadata_Treatment_Dose_Inhibitor_Dose,threeb_Metadata_Treatment_Dose_Inhibitor_Dose,fourb_Metadata_Treatment_Dose_Inhibitor_Dose
0,SH-SY5Y,B13,3765,6,Media ctr,0.0,None,media ctr,0.0,None,...,0.099225,0.096054,0.110432,0.107764,media ctr,0,media ctr_0_Media ctr_0.0,media ctr_Media ctr_0.0__0,media ctr__0__Media ctr_0.0,media ctr__0__Media ctr__0.0
1,SH-SY5Y,B13,3765,6,Media ctr,0.0,None,media ctr,0.0,None,...,0.020683,0.016122,0.012990,0.013631,media ctr,0,media ctr_0_Media ctr_0.0,media ctr_Media ctr_0.0__0,media ctr__0__Media ctr_0.0,media ctr__0__Media ctr__0.0
2,SH-SY5Y,B13,3765,6,Media ctr,0.0,None,media ctr,0.0,None,...,0.027519,0.045982,0.037653,0.017281,media ctr,0,media ctr_0_Media ctr_0.0,media ctr_Media ctr_0.0__0,media ctr__0__Media ctr_0.0,media ctr__0__Media ctr__0.0
3,SH-SY5Y,B13,3765,6,Media ctr,0.0,None,media ctr,0.0,None,...,-0.002945,0.000293,0.008048,-0.002151,media ctr,0,media ctr_0_Media ctr_0.0,media ctr_Media ctr_0.0__0,media ctr__0__Media ctr_0.0,media ctr__0__Media ctr__0.0
4,SH-SY5Y,B13,3765,6,Media ctr,0.0,None,media ctr,0.0,None,...,0.016846,0.006499,0.014114,0.026206,media ctr,0,media ctr_0_Media ctr_0.0,media ctr_Media ctr_0.0__0,media ctr__0__Media ctr_0.0,media ctr__0__Media ctr__0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597897,SH-SY5Y,O23,3555,6,Media ctr,0.0,None,media ctr,0.0,None,...,-0.024370,-0.033577,-0.033811,-0.028148,media ctr,0,media ctr_0_Media ctr_0.0,media ctr_Media ctr_0.0__0,media ctr__0__Media ctr_0.0,media ctr__0__Media ctr__0.0
597898,SH-SY5Y,O23,3555,6,Media ctr,0.0,None,media ctr,0.0,None,...,-0.049855,-0.048752,-0.050176,-0.049417,media ctr,0,media ctr_0_Media ctr_0.0,media ctr_Media ctr_0.0__0,media ctr__0__Media ctr_0.0,media ctr__0__Media ctr__0.0
597899,SH-SY5Y,O23,3555,6,Media ctr,0.0,None,media ctr,0.0,None,...,-0.027676,-0.024066,-0.026279,-0.027050,media ctr,0,media ctr_0_Media ctr_0.0,media ctr_Media ctr_0.0__0,media ctr__0__Media ctr_0.0,media ctr__0__Media ctr__0.0
597900,SH-SY5Y,O23,3555,6,Media ctr,0.0,None,media ctr,0.0,None,...,0.308702,0.309290,0.311365,0.309627,media ctr,0,media ctr_0_Media ctr_0.0,media ctr_Media ctr_0.0__0,media ctr__0__Media ctr_0.0,media ctr__0__Media ctr__0.0


In [5]:
# Combine treatment with dosage to be able to discern treatments with different doses as a different condition
# Combine treatment and dose
# df = df.assign(
# Metadata_Treatment_and_Dose=lambda x: df["Metadata_treatment"]
# + "_"
# + df["Metadata_dose"]
# )

# print("Unique Catagories are:")
# print(df["Metadata_Treatment_and_Dose"].unique())

# Generate df specific to analysis and model
# df = df.query(
# "Metadata_Treatment_and_Dose == 'LPS_10µg/ml'| Metadata_Treatment_and_Dose == 'Media only_0' | Metadata_Treatment_and_Dose == 'Disulfiram_2.5µM'"
# )
# for binary classification testing
# df = df.query(
#     "Metadata_Treatment_and_Dose == 'LPS_10µg/ml'| Metadata_Treatment_and_Dose == 'Media only_0'"
# )
# print("Selected Catagories are:")
# print(df["Metadata_Treatment_and_Dose"].unique())
# Drop na and reindex accordingly
# df = df.dropna()
# df = df.reset_index(drop=True)

# Check for Nans again
# df_stats(df)

# Understand categorical data such as treatment and dosing
# df[["Metadata_Treatment_and_Dose"]].drop_duplicates()

if params.DATA_SUBSET_OPTION == "True":
    df = df.sample(n=params.DATA_SUBSET_NUMBER)
    print("Data Subset Is On")
    print(f"Data is subset to {params.DATA_SUBSET_NUMBER}")
else:
    print("Data Subset Is Off")

# Code snippet for metadata extraction by Jenna Tomkinson
df_metadata = list(df.columns[df.columns.str.contains("Metadata")])

# define which columns are data and which are descriptive
df_descriptive = df[df_metadata]
df_values = df.drop(columns=df_metadata)

Data Subset Is Off


In [6]:
# Creating label encoder
le = preprocessing.LabelEncoder()
# Converting strings into numbers
df_values["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = le.fit_transform(
    df_descriptive["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
)
# split into X and Y where Y are the predictive column and x are the observable data
df_values_X = df_values.drop("oneb_Metadata_Treatment_Dose_Inhibitor_Dose", axis=1)
df_values_Y = df_values["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]

##### Regression Model Data Wrangling and Set Up
comment out if not using regression

In [7]:
# if params.DATA_SUBSET_OPTION == 'True':
#     df = df.sample(n=params.DATA_SUBSET_NUMBER)
#     print("yes")
# else:
#     pass
# df_stats(df)
# # Drop na and reindex accordingly
# df = df.dropna()
# df = df.reset_index(drop=True)

# # Check for Nans again
# df_stats(df)
# # Code snippet for metadata extraction by Jenna Tomkinson
# df_metadata = list(df.columns[df.columns.str.startswith("Metadata")])
# # define which columns are data and which are descriptive
# df_descriptive = df[df_metadata]
# df_values = df.drop(columns=df_metadata)
# df_values_Y = df_values['Nuclei_Texture_InverseDifferenceMoment_CorrER_3_01_256']
# df_values_X = df_values.drop('Nuclei_Texture_InverseDifferenceMoment_CorrER_3_01_256', axis=1)

#### Split Data - All Models can proceed through this point

In [8]:
X_train, X_test, X_val, Y_train, Y_test, Y_val = data_split(
    X_vals=df_values_X,
    y_vals=df_values_Y,
    train_proportion=0.8,
    val_proportion=0.1,
    test_proportion=0.1,
    seed=1,
    params=params,
)

In [9]:
# produce data objects for train, val and test datasets
train_data = Dataset_formatter(
    torch.FloatTensor(X_train.values), torch.FloatTensor(Y_train.values)
)
val_data = Dataset_formatter(
    torch.FloatTensor(X_val.values), torch.FloatTensor(Y_val.values)
)
test_data = Dataset_formatter(
    torch.FloatTensor(X_test.values), torch.FloatTensor(Y_test.values)
)

In [10]:
params.IN_FEATURES = X_train.shape[1]
print("Number of in features: ", params.IN_FEATURES)
if params.MODEL_TYPE == "Regression":
    params.OUT_FEATURES = 1
else:
    params.OUT_FEATURES = len(
        df_values["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].unique()
    )

print("Number of out features: ", params.OUT_FEATURES)

if params.OUT_FEATURES > 2:
    params.MODEL_TYPE = "Multi_Class"
elif params.OUT_FEATURES == 2:
    params.OUT_FEATURES = params.OUT_FEATURES - 1
    params.MODEL_TYPE = "Binary_Classification"
elif params.OUT_FEATURES == 1:
    params.MODEL_TYPE = "Regression"
else:
    pass
print(params.MODEL_TYPE)

Number of in features:  2847
Number of out features:  36
Multi_Class


In [11]:
# convert data class into a dataloader to be compatible with pytorch
train_loader = torch.utils.data.DataLoader(
    dataset=train_data, batch_size=params.BATCH_SIZE
)
valid_loader = torch.utils.data.DataLoader(
    dataset=val_data, batch_size=params.BATCH_SIZE
)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=1)

In [12]:
# no accuracy function must be loss for regression
if params.MODEL_TYPE == "Regression":
    params.METRIC = "loss"
    params.DIRECTION = "minimize"
else:
    pass


# wrap the objective function inside of a lambda function to pass args...
objective_lambda_func = lambda trial: objective_model_optimizer(
    train_loader,
    valid_loader,
    trial=trial,
    params=params,
    metric=params.METRIC,
    return_info=False,
)


# Study is the object for model optimization
study = optuna.create_study(direction=f"{params.DIRECTION}")
# Here I apply the optimize function of the study to the objective function
# This optimizes each parameter specified to be optimized from the defined search space
study.optimize(objective_lambda_func, n_trials=params.N_TRIALS)
# Prints out the best trial's optimized parameters
objective_model_optimizer(
    train_loader,
    valid_loader,
    trial=study.best_trial,
    params=params,
    metric=params.METRIC,
    return_info=True,
)

[I 2023-04-26 22:49:55,654] A new study created in memory with name: no-name-827f5fdb-8266-400c-80f2-a8882af1e527
[W 2023-04-26 22:51:08,946] Trial 0 failed with parameters: {'n_layers': 2, 'n_units_l0': 2, 'dropout_0': 0.16989255091927732, 'n_units_l1': 3, 'dropout_1': 0.1457547263524537, 'learning_rate': 0.04076261352883801, 'optimizer': 'Adam'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/lippincm/miniconda3/envs/Interstellar/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_384442/975347091.py", line 10, in <lambda>
    objective_lambda_func = lambda trial: objective_model_optimizer(
  File "/home/lippincm/Documents/ML/Interstellar_Analysis/MLP_Model/MLP_utils/utils.py", line 582, in objective_model_optimizer
    ) = train_n_validate(
  File "/home/lippincm/Documents/ML/Interstellar_Analysis/MLP_Model/MLP_utils/utils.py", line 370, in trai

KeyboardInterrupt: 

In [ ]:
fig = optuna.visualization.plot_optimization_history(study)
graph_path = f"./figures/{params.MODEL_TYPE}/plot_optimization_history_graph"
fig.write_html(Path(f"{graph_path}.html"))
fig.write_image(Path(f"{graph_path}.png"))
fig.show()

In [ ]:
fig = optuna.visualization.plot_intermediate_values(study)
graph_path = f"./figures/{params.MODEL_TYPE}/plot_intermediate_values_graph"
fig.write_html(Path(f"{graph_path}.html"))
fig.write_image(Path(f"{graph_path}.png"))
fig.show()

In [ ]:
# call function for best trial parameter extraction
param_dict = extract_best_trial_params(study.best_params)

In [ ]:
# call the optimized training model
train_loss, train_acc, valid_loss, valid_acc, epochs_ran, model = train_optimized_model(
    params.TRAIN_EPOCHS,
    train_loader,
    valid_loader,
    param_dict,
    params,
)
if params.MODEL_TYPE == "Regression":
    training_stats = pd.DataFrame(
        zip(train_loss, valid_loss, epochs_ran),
        columns=["train_loss", "valid_loss", "epochs_ran"],
    )
else:
    training_stats = pd.DataFrame(
        zip(train_loss, train_acc, valid_loss, valid_acc, epochs_ran),
        columns=["train_loss", "train_acc", "valid_loss", "valid_acc", "epochs_ran"],
    )

In [ ]:
if params.MODEL_TYPE == "Regression":
    pass
else:
    plot_metric_vs_epoch(
        training_stats,
        x="epochs_ran",
        y1="train_acc",
        y2="valid_acc",
        title="Accuracy vs. Epochs",
        x_axis_label="Epochs",
        y_axis_label="Accuracy",
        params=params,
    )

In [ ]:
plot_metric_vs_epoch(
    training_stats,
    x="epochs_ran",
    y1="train_loss",
    y2="valid_loss",
    title="Loss vs. Epochs",
    x_axis_label="Epochs",
    y_axis_label="Loss",
    params=params,
)

In [ ]:
# calling the testing function and outputting list values of tested model
if params.MODEL_TYPE == "Multi_Class" or params.MODEL_TYPE == "Regression":
    y_pred_list = test_optimized_model(model, test_loader, params)
elif params.MODEL_TYPE == "Binary_Classification":
    y_pred_list, y_pred_prob_list = test_optimized_model(model, test_loader, params)
else:
    raise Exception("Model type must be specified for proper model testing")


# un-nest list if nested i.e. length of input data does not match length of output data
if len(y_pred_list) != len(Y_test):
    y_pred_list = un_nest(y_pred_list)
    y_pred_prob_list = un_nest(y_pred_prob_list)
else:
    pass

In [ ]:
# Call visualization function
# calling the testing function and outputing list values of tested model
if params.MODEL_TYPE == "Multi_Class" or params.MODEL_TYPE == "Regression":
    confusion_matrix_df = results_output(y_pred_list, Y_test, params)
elif params.MODEL_TYPE == "Binary_Classification":
    results_output(y_pred_list, Y_test, params, y_pred_prob_list)
else:
    raise Exception("Model type must be specified for proper model testing")